In [111]:
import pandas as pd
import math

In [3]:
table = pd.read_csv('TABLE.csv', sep =";")
table

,DEPART,ARRIVEE,DISTANCE
0,A,B,3
1,A,C,4
2,A,D,8
3,B,C,4
4,B,E,9
5,C,D,4
6,C,E,5
7,D,E,7
8,D,F,4
9,E,G,8


In [93]:
names = [item for sublist in [list(table.DEPART) , list(table.ARRIVEE)]  for item in sublist]
names = sorted(list(set(names)))

adjmatrix = pd.DataFrame(columns = names)

for m in names:
    for n in names:
        if n in list(table[table["DEPART"] == m].ARRIVEE):
            adjmatrix.loc[m,n] = float(table[(table["DEPART"] == m) & (table["ARRIVEE"] == n)].DISTANCE)
        else:
            adjmatrix.loc[m,n] = 0
            
adjmatrix     

,A,B,C,D,E,F,G
A,0,3,4,8,0,0,0
B,0,0,4,0,9,0,0
C,0,0,0,4,5,0,0
D,0,0,0,0,7,4,0
E,0,0,0,0,0,0,8
F,0,0,0,0,0,0,10
G,0,0,0,0,0,0,0


In [116]:
adlist = []
start = "A"

for m in names:
    if m == start:
        sublist = [m, 0]
    else:
        sublist = [m, math.inf]
    
    for n in names:
        if adjmatrix.loc[m, n] > 0:
            sublist.append([n, adjmatrix.loc[m, n]])  
    if len(sublist) > 2:
        adlist.append(sublist)
adlist


[['A', 0, ['B', 3.0], ['C', 4.0], ['D', 8.0]],
 ['B', inf, ['C', 4.0], ['E', 9.0]],
 ['C', inf, ['D', 4.0], ['E', 5.0]],
 ['D', inf, ['E', 7.0], ['F', 4.0]],
 ['E', inf, ['G', 8.0]],
 ['F', inf, ['G', 10.0]]]

In [123]:
adlist[1][2:][0][0]

'C'

In [132]:
# mapper1

visited_table = []
current_node = start
current_dist = []

for l in adlist:
    if l[0] == current_node:
        visited_table.append(l[2:])
    else:
        var_list = []
        for j in range(len(l[2:])):
            var_list. append([l[2:][j][0], math.inf])
        current_dist.append(var_list)
            
print(current_dist)
print(visited_table)

[[['C', inf], ['E', inf]], [['D', inf], ['E', inf]], [['E', inf], ['F', inf]], [['G', inf]], [['G', inf]]]
[[['B', 3.0], ['C', 4.0], ['D', 8.0]]]


In [137]:
[m[0] for m in visited_table[0]]

['B', 'C', 'D']

In [138]:
# reducer 1
k = 1 # itération 1

for l in current_dist:
    for j in l:
        if j[0] in [m[0] for m in visited_table[k-1]]:
            print(j)



['C', inf]
['D', inf]


In [150]:
red_output = []

for l in adlist:
    for m in visited_table[0]:
        if l[0] == m[0]:
            l[1] = m[1]
            for i in range(len(l[2:])):
                l[2:][i][1] = l[2:][i][1] + m[1]
                
    red_output.append(l)

red_output

[['A', 0, ['B', 3.0], ['C', 4.0], ['D', 8.0]],
 ['B', 3.0, ['C', 7.0], ['E', 12.0]],
 ['C', 4.0, ['D', 8.0], ['E', 9.0]],
 ['D', 8.0, ['E', 15.0], ['F', 12.0]],
 ['E', inf, ['G', 8.0]],
 ['F', inf, ['G', 10.0]]]

In [ ]:
## MAPPER 2
# on veut avoir la liste des nodes parcourus, et garder les plus petits pour chaque point à partir du node d'avant

current_step = 2 # on a parcouru tous les nodes qui étaient à 1 vecteur de start, maintenant on veut append les distances